In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

In [ ]:
url="https://media.githubusercontent.com/media/PacktPublishing/Advanced-NLP-Projects-with-TensorFlow-2.0/master/section_5_notebooks/reviews.csv"

In [ ]:
path=tf.keras.utils.get_file('reviews.csv',origin=url)

300908544/300904694 [==============================] - 2s 0us/step


In [ ]:
train=pd.read_csv(path)

In [ ]:
train['Text'][0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [ ]:
train = train[['Summary','Text']]


In [ ]:
train

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...
...,...,...
568449,Will not do without,Great for sesame chicken..this is a good if no...
568450,disappointed,I'm disappointed with the flavor. The chocolat...
568451,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [ ]:
train['text_length'] = train['Text'].str.count(' ')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
train['text_length'].describe()

count    568454.000000
mean         81.005522
std          80.807102
min           2.000000
25%          33.000000
50%          57.000000
75%          99.000000
max        3525.000000
Name: text_length, dtype: float64

In [ ]:
train['summary_length'] = train['Summary'].str.count(' ')
train['summary_length'].describe()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


count    568427.000000
mean          3.128462
std           2.619420
min           0.000000
25%           1.000000
50%           3.000000
75%           4.000000
max          41.000000
Name: summary_length, dtype: float64

In [ ]:
train = train.loc[train['summary_length']<8]


In [ ]:
train = train.loc[train['text_length']<30]

In [ ]:
train.shape

(109053, 4)

In [ ]:
train

In [ ]:
train['text_lower']=train['Text'].str.lower()

In [ ]:
train['text_no_punctuation']=train['text_lower'].str.replace('[^\w\s]','')
    

In [ ]:
train['text_no_punctuation']

In [ ]:
train['summary_lower']=train["Summary"].str.lower()
train['summary_no_punctuation'] =  '_start_' + ' ' +train['summary_lower'].str.replace('[^\w\s]','')+ ' ' +'_end_'


In [ ]:
train['summary_no_punctuation'] 

In [ ]:
max_features1=5000
maxlen1=30
max_features2=5000
maxlen2=8

In [ ]:
tok1 = tf.keras.preprocessing.text.Tokenizer(num_words=max_features1) 
tok1.fit_on_texts(list(train['text_no_punctuation'].astype(str))) #fit to cleaned text
tf_train_text =tok1.texts_to_sequences(list(train['text_no_punctuation'].astype(str)))
tf_train_text =tf.keras.preprocessing.sequence.pad_sequences(tf_train_text, maxlen=maxlen1) 

In [ ]:
tok2 = tf.keras.preprocessing.text.Tokenizer(num_words=max_features2, filters = '*') 
tok2.fit_on_texts(list(train['summary_no_punctuation'].astype(str))) #fit to cleaned text
tf_train_summary = tok2.texts_to_sequences(list(train['summary_no_punctuation'].astype(str)))
tf_train_summary = tf.keras.preprocessing.sequence.pad_sequences(tf_train_summary, maxlen=maxlen2, padding ='post')


In [ ]:
vectorized_summary=tf_train_summary


In [ ]:
decoder_input_data=vectorized_summary[:,:-1]

In [ ]:
decoder_target_data=vectorized_summary[:,1:]

In [ ]:
decoder_target_data.shape

(109053, 7)

In [ ]:
decoder_input_data.shape

(109053, 7)

In [ ]:
vectorized_text = tf_train_text

In [ ]:
encoder_input_data = vectorized_text
doc_length = encoder_input_data.shape[1]

In [ ]:
encoder_input_data.shape

(109053, 30)

In [ ]:
doc_length

30

In [ ]:
vocab_size_encoder = len(tok1.word_index) + 1 
vocab_size_decoder = len(tok2.word_index) + 1


In [ ]:
latent_dim=256

In [ ]:
encoder_inputs = tf.keras.Input(shape=(doc_length,), name='Encoder-Input')
x = tf.keras.layers.Embedding(vocab_size_encoder, latent_dim, name='Body-Word-Embedding', mask_zero=False)(encoder_inputs)
x = tf.keras.layers.BatchNormalization(name='Encoder-Batchnorm-1')(x)
_, state_h = tf.keras.layers.GRU(50, return_state=True, name='Encoder-Last-GRU')(x)
encoder_model = tf.keras.Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')
seq2seq_encoder_out = encoder_model(encoder_inputs)
decoder_inputs = tf.keras.Input(shape=(None,), name='Decoder-Input') 
dec_emb = tf.keras.layers.Embedding(vocab_size_decoder, latent_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
dec_bn = tf.keras.layers.BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)
decoder_gru = tf.keras.layers.GRU(50, return_state=True, return_sequences=True, name='Decoder-GRU')
decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out) 
x = tf.keras.layers.BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)
decoder_dense = tf.keras.layers.Dense(vocab_size_decoder, activation='softmax', name='Final-Output-Dense')
decoder_outputs = decoder_dense(x)
seq2seq_Model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
seq2seq_Model.compile(optimizer=tf.keras.optimizers.Nadam(lr=0.001), loss='sparse_categorical_crossentropy')

In [ ]:
batch_size = 64
epochs = 2 
history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,  epochs=epochs ,  validation_split=0.12)

Epoch 1/2
1500/1500 [==============================] - 205s 137ms/step - loss: 2.6240 - val_loss: 1.9304
Epoch 2/2
1500/1500 [==============================] - 203s 135ms/step - loss: 1.7349 - val_loss: 1.7094
